In [2]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [3]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [4]:
from __future__ import print_function
from numpy import *
from scipy import *
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import scipy as sp
import xarray as xr

import seawater
import datetime as dt
""
from salishsea_tools import (
    nc_tools,
    viz_tools,
    geo_tools,
    tidetools
)

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
plt.style.use('seaborn-whitegrid')
import netCDF4 as nc

import cmocean as cm
import glob
import sys
sys.path.append('/data/tjarniko/mocsy')
sys.path.append('/data/tjarniko/MEOPAR/at3/notebooks/carbon_dev/CCCmaDEV/CCCma_src')
import mocsy
import CCCma
import CCCma_stations as cs
from matplotlib import reload
import arrow
import gsw
%matplotlib inline

/home/tjarniko/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [5]:
sdir = '/data/tjarniko/results/BR_2nd_2015_cop/SKOG_2/ncs/'
sdir3 = '/data/tjarniko/results/PREIND_2nd_2015/PI_2/ncs/'


start = '2015-01-01'
end = '2015-12-31'
start3 = '2015-01-01'
end3 = '2015-12-31'

st = dt.datetime(2015,1,1)
en = dt.datetime(2015,12,31)
st3 = dt.datetime(2015,1,1)
en3 = dt.datetime(2015,12,31)


y_st = st.timetuple().tm_yday
print(y_st)
y_en = en.timetuple().tm_yday
print(y_en)
ts_BR = np.arange(y_st,y_en+1,1)

#PI year
y_st3 = st3.timetuple().tm_yday
print(y_st3)
y_en3 = en3.timetuple().tm_yday
print(y_en3)
ts_PI = np.arange(y_st3,y_en3+1,1)
days_in = np.size(ts_PI)


1
365
1
365


In [6]:
def make_nclen(start,end,ftype, sdir):
    base_ar = []
    sens_ar = []
    start_run = arrow.get(start)
    end_run = arrow.get(end)
    arrow_array = []
    for r in arrow.Arrow.span_range('day', start_run, end_run):
        arrow_array.append(r)

    dayslen = len(arrow_array)
    for i in range(0,dayslen):
        tdate = arrow_array[i][0]
        ddmmmyy = tdate.format('DDMMMYY').lower()
        ymd = tdate.format('YYYYMMDD')
        nc_sens = sdir + '/SKOG_1d_*'+ ftype +'*' + ymd + '-' + ymd + '.nc'
        tnc_sens = glob.glob(nc_sens)
        sens_ar.append(tnc_sens[0])
    return sens_ar

def make_nclen_transport(start,end,sdir):
    base_ar = []
    sens_ar = []
    start_run = arrow.get(start)
    end_run = arrow.get(end)
    arrow_array = []
    for r in arrow.Arrow.span_range('day', start_run, end_run):
        arrow_array.append(r)

    dayslen = len(arrow_array)
    for i in range(0,dayslen):
        tdate = arrow_array[i][0]
        ddmmmyy = tdate.format('DDMMMYY').lower()
        ymd = tdate.format('YYYYMMDD')
        nc_sens = sdir + '/SKOG_1d_*' +'dian_U_' + ymd + '-' + ymd + '.nc'
        tnc_sens = glob.glob(nc_sens)
        sens_ar.append(tnc_sens[0])

    return sens_ar

def make_nclen_transport_V(start,end,sdir):
    base_ar = []
    sens_ar = []
    start_run = arrow.get(start)
    end_run = arrow.get(end)
    arrow_array = []
    for r in arrow.Arrow.span_range('day', start_run, end_run):
        arrow_array.append(r)

    dayslen = len(arrow_array)
    for i in range(0,dayslen):
        tdate = arrow_array[i][0]
        ddmmmyy = tdate.format('DDMMMYY').lower()
        ymd = tdate.format('YYYYMMDD')
        nc_sens = sdir + '/SKOG_1d_*' +'dian_V_' + ymd + '-' + ymd + '.nc'
        tnc_sens = glob.glob(nc_sens)
        sens_ar.append(tnc_sens[0])

    return sens_ar

In [7]:
BR_ar = make_nclen(start,end,'carp', sdir)
PI_ar = make_nclen(start3,end3,'carp', sdir3)
BR_ar_tp = make_nclen_transport(start,end, sdir)
PI_ar_tp = make_nclen_transport(start3,end3,sdir3)

BR_ar_tpV = make_nclen_transport_V(start,end, sdir)
PI_ar_tpV = make_nclen_transport_V(start3,end3,sdir3)

In [8]:
grid = nc.Dataset('/data/tjarniko/MEOPAR/grid/mesh_mask201702.nc')
vdir = grid['e2t'][0,0:878,20:398]
udir = grid['e1t'][0,0:878,20:398]
wdir = grid['e3t_0'][0,:,0:878,20:398]
wdir_20 = grid['e3t_0'][0,0:20,0:878,20:398]
wdir_20_100 = grid['e3t_0'][0,20:27,0:878,20:398]
wdir_deep = grid['e3t_0'][0,27:40,0:878,20:398]
surfa = vdir*udir
size_domain = wdir *surfa
size_domain_20 = wdir_20 *surfa
size_domain_20_100 = wdir_20_100 *surfa
size_domain_deep = wdir_deep *surfa


In [33]:
def calculate_total_C(tdat, size_domain):
    tcar = tdat['dissolved_inorganic_carbon'][:]
    shape_ar = tcar.shape
    days_in = shape_ar[0]
    stor_mol = np.zeros(days_in)

    for i in range(0, days_in):
        if i%10 == 0:
            print(i)
        var_tmp = tdat['dissolved_inorganic_carbon'][i,:,0:878,20:398]
        loaded_vt = var_tmp.values
#       var_tmp_loaded = var_tmp.load_data()
        loaded_vt[loaded_vt == 1e+20] = 0
        loaded_vt2 = loaded_vt * size_domain
        totdic = np.sum(np.sum(loaded_vt2))
        totdic_mols = totdic * (1/1000)        
        stor_mol[i] = totdic_mols
    return stor_mol



In [34]:
stor_mol = calculate_total_C(tdat, size_domain)

0
10
20
30
40
50
60
70
80
90


KeyboardInterrupt: 

In [11]:
tdat = xr.open_mfdataset(BR_ar)


<xarray.DataArray 'time_counter' (time_counter: 365)>
array(['2015-01-01T12:00:00.000000000', '2015-01-02T12:00:00.000000000',
       '2015-01-03T12:00:00.000000000', ..., '2015-12-29T12:00:00.000000000',
       '2015-12-30T12:00:00.000000000', '2015-12-31T12:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
    time_centered  (time_counter) datetime64[ns] 2015-01-01T12:00:00 ...
  * time_counter   (time_counter) datetime64[ns] 2015-01-01T12:00:00 ...
Attributes:
    axis:           T
    standard_name:  time
    long_name:      Time axis
    time_origin:    1900-01-01 00:00:00
    bounds:         time_counter_bounds


In [16]:
tcar = tdat['dissolved_inorganic_carbon'][:]
shape_ar = tcar.shape
days_in = shape_ar[0]
print(days_in)

365


In [11]:
car = tdat['dissolved_inorganic_carbon']

In [12]:
print(car)

<xarray.DataArray 'dissolved_inorganic_carbon' (time_counter: 31, deptht: 40, y: 898, x: 398)>
dask.array<concate..., shape=(31, 40, 898, 398), dtype=float64, chunksize=(1, 40, 898, 398)>
Coordinates:
  * deptht         (deptht) float32 0.5 1.5 2.50001 3.50003 4.50007 5.50015 ...
    nav_lat        (y, x) float32 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 ...
    nav_lon        (y, x) float32 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 ...
    time_centered  (time_counter) datetime64[ns] 2015-01-01T12:00:00 ...
  * time_counter   (time_counter) datetime64[ns] 2015-01-01T12:00:00 ...
Dimensions without coordinates: y, x
Attributes:
    standard_name:       mole_concentration_of_dissolved_inorganic_carbon_in_...
    long_name:           Dissolved Inorganic C Concentration
    units:               mmol m-3
    online_operation:    average
    interval_operation:  40 s
    interval_write:      1 d
    cell_methods:        time: mean (interval: 40 s)
    cell_measures:       area: area
